# Laboratorio 5 

##### Ma. Isabel Ortiz Naranjo
##### Luis Urbina

In [11]:
from scapy.all import *
import pandas as pd 
import numpy as np 
import binascii
import seaborn as sns
sns.set(color_codes = True)
%matplotlib inline

Captura de 100 paquetes

In [12]:
num_of_packets_to_sniff = 100
pcap = sniff(count = num_of_packets_to_sniff)
# rdpcap returns packet list
## packetlist object can be enumerated 
print("Tipo:", type(pcap))
print("Largo: ", len(pcap))
print(pcap)

PermissionError: [Errno 1] Operation not permitted

TCP

In [7]:
"""
0                   1                   2                   3
0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9 0 1
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
|          Source Port          |       Destination Port        |
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
|                        Sequence Number                        |
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
|                    Acknowledgment Number                      |
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
|  Data |           |U|A|P|R|S|F|                               |
| Offset| Reserved  |R|C|S|S|Y|I|            Window             |
|       |           |G|K|H|T|N|N|                               |
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
|           Checksum            |         Urgent Pointer        |
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
|                    Options                    |    Padding    |
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
|                             data                              |
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
02 04 05 a0 01 03 03 05 01 01 08 0a 1d 74 65 c5 00 00 00 00 04 02 00 00
"""

print()

Exploración

In [6]:
#Recordemos que en los modelos de red, las capas inferiores encapsulan la información de las capas superiores
#Frames en paquetes, paquetes en segmentos (Ethernet, IP), modelo TCP/IP

ethernet_frame = pcap[0]
ip_packet = ethernet_frame.payload
segment = ip_packet.payload
data = segment.payload #recupera la informacion de la capa 4

print("Informacion del frame:")
print(ethernet_frame.summary())

print("Informacion del paquete:")
print(ip_packet.summary())

print("Informacion del segmento:")
print(segment.summary())

print("Data:")
print(data.summary())

ethernet_frame.show()
data.show()

Informacion del frame:
Ether / IP / TCP 172.28.0.2:8080 > 172.28.0.1:52420 FA
Informacion del paquete:
IP / TCP 172.28.0.2:8080 > 172.28.0.1:52420 FA
Informacion del segmento:
TCP 172.28.0.2:8080 > 172.28.0.1:52420 FA
Data:

###[ Ethernet ]### 
  dst       = 02:42:04:54:f1:51
  src       = 02:42:ac:1c:00:02
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 52
     id        = 9466
     flags     = DF
     frag      = 0
     ttl       = 64
     proto     = 6
     chksum    = 0xbd8e
     src       = 172.28.0.2
     dst       = 172.28.0.1
     \options   \
###[ TCP ]### 
        sport     = 8080
        dport     = 52420
        seq       = 293036563
        ack       = 2738177287
        dataofs   = 8
        reserved  = 0
        flags     = FA
        window    = 501
        chksum    = 0x5862
        urgptr    = 0
        options   = [('NOP', None), ('NOP', None), ('Timestamp', (2326413667, 1059821441))]



Tipos de datos en Scapy

In [7]:
print(type(ethernet_frame))
print(type(ip_packet))
print(type(segment))

# Packets can be filtered on layers ie) ethernet_frame[scapy.layers.l2.Ether]
ethernet_type = type(ethernet_frame)
ip_type = type(ip_packet)
tcp_type = type(segment)
print("Ethernet",pcap[ethernet_type])
print("IP", pcap[ip_type])
print("TCP", pcap[tcp_type])

# Scapy provides this via import statements
from scapy.layers.l2 import Ether
from scapy.layers.inet import IP
from scapy.layers.inet import TCP, UDP

print("UDP", pcap[UDP])

<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.inet.IP'>
<class 'scapy.layers.inet.TCP'>
Ethernet <Ether from Sniffed: TCP:10 UDP:0 ICMP:0 Other:0>
IP <IP from Sniffed: TCP:10 UDP:0 ICMP:0 Other:0>
TCP <TCP from Sniffed: TCP:10 UDP:0 ICMP:0 Other:0>
UDP <UDP from Sniffed: TCP:0 UDP:0 ICMP:0 Other:0>


Conversion de PCAP a DF

In [8]:
# Obtenemos los campos de IP/TCP/UDP que seran las columnas de nuestro DF
ip_fields = [field.name for field in IP().fields_desc]
tcp_fields = [field.name for field in TCP().fields_desc]
udp_fields = [field.name for field in UDP().fields_desc]

#Concatenamos todos los campos
dataframe_fields = ip_fields + ['time'] + tcp_fields + ['payload','payload_raw','payload_hex']

# Creamos un DF en blanco con los campos como columnas
df = pd.DataFrame(columns=dataframe_fields)
for packet in pcap[IP]:
    # Obtenemos los valores de los campos de cada fila
    field_values = []
    # A;adimos todos los valores de los campos de IP al df
    for field in ip_fields:
        if field == 'options':
            # Obtenemos el numero de opciones del IP Header
            field_values.append(len(packet[IP].fields[field]))
        else:
            field_values.append(packet[IP].fields[field])
    
    field_values.append(packet.time)
    
    layer_type = type(packet[IP].payload)
    for field in tcp_fields:
        try:
            if field == 'options':
                field_values.append(len(packet[layer_type].fields[field]))
            else:
                field_values.append(packet[layer_type].fields[field])
        except:
            field_values.append(None)
    
    # Agregamos el payload
    field_values.append(len(packet[layer_type].payload))
    field_values.append(packet[layer_type].payload.original)
    field_values.append(binascii.hexlify(packet[layer_type].payload.original))
    # Agregamos la fila al df
    df_append = pd.DataFrame([field_values], columns=dataframe_fields)
    df = pd.concat([df, df_append], axis=0)

# Reset Index
df = df.reset_index()
# Drop old index column
df = df.drop(columns="index")

print(df.head(5))

  version ihl tos  len     id flags frag ttl proto chksum  ... dataofs  \
0       4   5   0   52   9466    DF    0  64     6  48526  ...       8   
1       4   5   0   52  38954    DF    0  64     6  19038  ...       8   
2       4   5   0  878  29373    DF    0  64     6  27791  ...       8   
3       4   5   0   52  56910    DF    0  64     6   1080  ...       8   
4       4   5   0  878  52462    DF    0  64     6   4702  ...       8   

  reserved flags  window chksum urgptr options payload  \
0        0    FA     501  22626      0       3       0   
1        0     A     501  22626      0       3       0   
2        0    PA     507  23454      0       3     826   
3        0     A     501  22628      0       3       0   
4        0    PA     507  23454      0       3     826   

                                         payload_raw  \
0                                                b''   
1                                                b''   
2  b'\x81~\x036{"header": {"msg_id": "

In [9]:
# Return Src Address, Dst Address, Src Port, Dst Port
df[['src','dst','sport','dport']]

,src,dst,sport,dport
0,172.28.0.2,172.28.0.1,8080,52420
1,172.28.0.1,172.28.0.2,52420,8080
2,172.28.0.2,172.28.0.3,9000,36854
3,172.28.0.3,172.28.0.2,36854,9000
4,172.28.0.3,172.28.0.2,6000,58480
5,172.28.0.2,172.28.0.3,58480,6000
6,172.28.0.1,172.28.0.2,52488,8080
7,172.28.0.2,172.28.0.1,8080,52488
8,172.28.0.2,172.28.0.1,8080,52488
9,172.28.0.1,172.28.0.2,52488,8080


In [10]:
#Calcular estadisticas
print("# Top Source Address")
print(df['src'].describe(),'\n\n')

# Top Source Address
count             10
unique             3
top       172.28.0.2
freq               5
Name: src, dtype: object 




In [12]:
#¿Quien es la IP de origen mas frecuente?
frequent_address = df['src'].describe()['top']


# A que IPs les habla la IP mas frecuente
print(df[df['src'] == frequent_address]['dst'].unique(),"\n\n")

['172.28.0.1' '172.28.0.3'] 


